In [1]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import trimesh as tm 
import tkinter as tk
from tkinter import filedialog 
import pandas as pd
import os
import seaborn as sns
from utilities import select_files, select_file, plot_all

bounding box

In [2]:

box = select_files()
mesh_b = pv.read(box[0].name)
plotter = pv.Plotter()
_ = plotter.add_mesh(mesh_b, opacity=0.70)
#mesh_holes['Stress GPa'] = sim_data['ver_stress']
#plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:53164/index.html?ui=P_0x210eca47da0_0&reconnect=auto" class="pyvis…

In [3]:
voxels = pv.voxelize(mesh_b, density=10)
p = pv.Plotter()
p.add_mesh(voxels, color=True, show_edges=True, opacity=0.5)
p.add_mesh(mesh_b, color="lightblue", opacity=0.3)
p.show()

Widget(value='<iframe src="http://localhost:53164/index.html?ui=P_0x210ecc09880_1&reconnect=auto" class="pyvis…

In [4]:
#open filtered list of meshes and simulation results  
filtered_files = select_file()
sim_files = pd.read_csv(filtered_files.name)


In [5]:
mesh_list = sim_files['sur_mesh']

In [7]:
#Show parts inside voxelized volume
p1 = pv.Plotter()
for i in range(len(mesh_list)):

    filename = mesh_list[i]
    mesh_i = pv.read(filename)
    p1.add_mesh(mesh_i, opacity=0.10, color=True)
p1.add_mesh(voxels, color=True, show_edges=True, opacity=0.5)
p1.show(cpos='yx' )

Widget(value='<iframe src="http://localhost:53164/index.html?ui=P_0x211b3e41880_3&reconnect=auto" class="pyvis…

In [8]:
print(voxels)

UnstructuredGrid (0x210f30337c0)
  N Cells:    1680
  N Points:   2184
  X Bounds:   -4.670e+01, 7.330e+01
  Y Bounds:   -1.800e+02, 2.000e+01
  Z Bounds:   -3.563e+00, 6.644e+01
  N Arrays:   2


In [36]:
n = voxels.n_cells
print(n)
print(voxels.bounds)


1680
(-46.703983306884766, 73.29601669311523, -180.0, 20.0, -3.5630009174346924, 66.43699908256531)


(-46.703983306884766,
 73.29601669311523,
 -180.0,
 20.0,
 -3.5630009174346924,
 66.43699908256531)

In [10]:
voxels.get_cell(0)

Cell (0x211ac156260)
  Type:        <CellType.HEXAHEDRON: 12>
  Linear:      True
  Dimension:   3
  N Points:    8
  N Faces:     6
  N Edges:     12
  X Bounds:    -4.670e+01, -3.670e+01
  Y Bounds:    -1.800e+02, -1.700e+02
  Z Bounds:    -3.563e+00, 6.437e+00

In [60]:
voxels.get_cell(1)

Cell (0x212bcc58b80)
  Type:        <CellType.HEXAHEDRON: 12>
  Linear:      True
  Dimension:   3
  N Points:    8
  N Faces:     6
  N Edges:     12
  X Bounds:    -3.670e+01, -2.670e+01
  Y Bounds:    -1.800e+02, -1.700e+02
  Z Bounds:    -3.563e+00, 6.437e+00

In [11]:
cell_0 = voxels.get_cell(0)
cell_01 = voxels.extract_cells(555)

In [12]:
m = pv.read(mesh_list[0])
intersection, s1_split, s2_split = m.intersection(voxels.extract_cells(555))

ERROR:root:Input for connection index 0 on input port index 1 for algorithm vtkIntersectionPolyDataFilter (000002130654D480) is of type vtkUnstructuredGrid, but a vtkPolyData is required.


Convert voxel to polydata(mesh like) structure to use intersection and volume methods

In [13]:
voxel_polydata = cell_01.extract_surface().triangulate()
intersection, s1_split, s2_split = m.intersection(voxel_polydata)
intersection_bool = voxel_polydata.boolean_intersection(m)

In [14]:
print(intersection_bool.points)

[[  -6.7039833 -120.          16.436998 ]
 [ -16.703983  -120.          26.432093 ]
 [ -16.703983  -120.          16.436998 ]
 [ -16.700638  -120.          26.433653 ]
 [  -6.7039833 -120.          26.436998 ]
 [ -16.693457  -120.          26.436998 ]
 [  -7.0060153 -119.788895    16.436998 ]
 [  -6.9172587 -119.78673     16.436998 ]
 [  -8.421813  -119.840416    16.436998 ]
 [  -8.6596985 -119.846725    16.436998 ]
 [  -9.018234  -119.859406    16.436998 ]
 [ -10.362341  -119.88914     16.436998 ]
 [ -12.122328  -119.954796    16.436998 ]
 [ -12.199137  -119.95667     16.436998 ]
 [ -12.221263  -119.94906     16.436998 ]
 [ -12.337474  -119.90748     16.436998 ]
 [  -6.7039833 -119.78152     16.436998 ]
 [ -16.703983  -114.030716    16.436998 ]
 [ -16.703983  -114.03437     16.757618 ]
 [ -16.703983  -113.83263     17.073717 ]
 [ -16.703983  -113.09008     18.166935 ]
 [ -16.703983  -112.8217      18.36518  ]
 [ -16.703983  -112.4431      18.55247  ]
 [ -16.703983  -112.209595    18.6

In [16]:
print(intersection_bool.volume)
print(voxel_polydata.volume)

821.9863731387766
1000.0


In [17]:
#c0 = voxels.get_cell(0).extract_surface().triangulate().boolean_intersection(m)
intersection_bool_0 = voxels.extract_cells(0).extract_surface().triangulate().boolean_intersection(m)

ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
d:\checkouts\ML-Training\ML-env\DS-ML\Lib\site-packages\pyvista\core\filters\poly_data.py:262: UserWarning: Unable to compute boolean intersection when one PolyData is contained within another and no faces intersect.
  warnings.warn(


In [18]:
print(intersection_bool_0.points)

[]


In [19]:
print(intersection.points)

[[  -9.140719  -110.          21.73365  ]
 [  -7.35421   -110.          21.716738 ]
 [  -7.276777  -110.          21.715553 ]
 [  -6.7039833 -110.32705     21.568653 ]
 [  -6.7039833 -111.22135     21.18207  ]
 [  -7.244628  -110.          21.715244 ]
 [  -6.7039833 -111.90925     20.870426 ]
 [  -6.7039833 -110.          21.706783 ]
 [ -11.659593  -110.          21.758022 ]
 [ -11.526387  -110.          21.767675 ]
 [  -9.675495  -110.          21.739603 ]
 [ -11.999922  -110.          21.732937 ]
 [  -6.7039833 -112.69983     20.50174  ]
 [  -6.7039833 -113.38739     20.16778  ]
 [  -6.7039833 -113.61645     20.053446 ]
 [ -16.700638  -120.          26.433653 ]
 [ -16.693457  -120.          26.436998 ]
 [ -16.703983  -119.65299     26.436998 ]
 [ -16.703983  -119.99516     26.432161 ]
 [ -16.703983  -111.53821     18.917223 ]
 [ -16.703983  -110.756485    19.217243 ]
 [ -16.703983  -110.174774    19.321833 ]
 [ -16.703983  -110.          19.352123 ]
 [ -16.703983  -112.209595    18.6

In [20]:
p2 = pv.Plotter()
p2.add_mesh(intersection_bool, color="red", opacity=1)
p2.add_mesh(voxels, color="lightblue", show_edges=True, opacity=0.1)
p2.add_mesh(m,color="green", opacity=0.3)
p2.show()

Widget(value='<iframe src="http://localhost:53164/index.html?ui=P_0x2120f72bb60_4&reconnect=auto" class="pyvis…

Compute the intersections and the intersection volume for all voxels
one mesh

In [22]:
def compute_voxel_intersections(surface_mesh, voxel_grid):
    n = voxel_grid.n_cells
    cell_percentage = np.zeros(n) 

    volume_0 = voxels.extract_cells(0).extract_surface().triangulate().volume
    
    for i in range(n):
        intersection = voxel_grid.extract_cells(i).extract_surface().triangulate().boolean_intersection(surface_mesh)
        if len(intersection.points)>0:
            cell_vol = intersection.volume/volume_0
            cell_percentage[i] = cell_vol
    
    return cell_percentage



In [23]:
p_list = compute_voxel_intersections(m, voxels)

ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
d:\checkouts\ML-Training\ML-env\DS-ML\Lib\site-packages\pyvista\core\filters\poly_data.py:262: UserWarning: Unable to compute boolean intersection when one PolyData is contained within another and no faces intersect.
  warnings.warn(
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells to operate on
ERROR:root:No points/cells t

In [32]:
p2 = pv.Plotter()
#p2.add_mesh(intersection_bool, color="red", opacity=1)
p2.add_mesh(voxels, color="lightblue", show_edges=True, opacity=0.1)
p2.add_mesh(m,color="green", opacity=0.3)
for i in range(len(p_list)):
    if(p_list[i]>0):
        if (p_list[i]>1):
            print(i, ', ', p_list[i])
            p_list[i] = p_list[i]/2
        p2.add_mesh(voxels.extract_cells(i), color="red", opacity=p_list[i])

p2.show()

295 ,  1.5482381895358015
442 ,  1.0445498375431277
543 ,  1.0003659533241989
546 ,  1.004975933156588
604 ,  1.1017622969128416
608 ,  1.170350712519928
649 ,  1.2717930183438646
650 ,  1.0858196116397367
657 ,  1.3548757899985198
809 ,  1.0014824469847625
860 ,  1.3984753500124825


Widget(value='<iframe src="http://localhost:53164/index.html?ui=P_0x211b3e41100_9&reconnect=auto" class="pyvis…

In [24]:
print(p_list)

[0. 0. 0. ... 0. 0. 0.]


In [25]:
inters_bool_0 = voxels.extract_cells(295).extract_surface().triangulate().boolean_intersection(m)
p5 = pv.Plotter()
p5.add_mesh(inters_bool_0, color="red", opacity=1)
p5.add_mesh(voxels, color="lightblue", show_edges=True, opacity=0.1)
p5.add_mesh(m,color="green", opacity=0.3)
p5.show()

Widget(value='<iframe src="http://localhost:53164/index.html?ui=P_0x212bc9f5430_5&reconnect=auto" class="pyvis…

In [26]:
inters_bool_0

PolyData (0x21193634e80)
  N Cells:    414
  N Points:   348
  N Strips:   0
  X Bounds:   2.330e+01, 3.330e+01
  Y Bounds:   -1.400e+02, -1.300e+02
  Z Bounds:   6.437e+00, 1.722e+01
  N Arrays:   6

In [27]:
inters_bool_0.volume

1548.2381895358014

In [28]:
b2 = voxels.extract_cells(295).extract_surface().triangulate().boolean_intersection(inters_bool_0)

ERROR:root:Algorithm vtkIntersectionPolyDataFilter (000002130655C390) returned failure for request: vtkInformation (00000212BE17C2A0)
ERROR:root:Algorithm vtkBooleanOperationPolyDataFilter (0000021236EE8740) returned failure for request: vtkInformation (00000212BE17C540)
ERROR:root:Algorithm vtkIntersectionPolyDataFilter (000002130655E150) returned failure for request: vtkInformation (00000212BE17C9A0)


In [29]:
inters_bool_0, s1, s2 = voxels.extract_cells(295).extract_surface().triangulate().intersection(m)
p5 = pv.Plotter()
p5.add_mesh(inters_bool_0, color="red", opacity=1)
p5.add_mesh(voxels, color="lightblue", show_edges=True, opacity=0.1)
p5.add_mesh(m,color="green", opacity=0.3)
p5.show()

Widget(value='<iframe src="http://localhost:53164/index.html?ui=P_0x211b51578c0_6&reconnect=auto" class="pyvis…

Voxel grid bounds
  X Bounds:   -4.670e+01, 7.330e+01
  Y Bounds:   -1.800e+02, 2.000e+01
  Z Bounds:   -3.563e+00, 6.644e+01

In [37]:
#ranges for bins
r_x = (-4.670e+01, 7.330e+01)
r_y = (-1.800e+02, 2.000e+01)
r_z = (3.563e+00, 6.644e+01)

In [39]:
# extract sim results list
sim_results_list = sim_files['sim_csv']
#read csv for mesh 0
sim_output = pd.read_csv(sim_results_list[0])


In [57]:
X = sim_output['x']
Y = sim_output['y']
Z = sim_output['z']
h_stress = sim_output['hor_stress']

#  Weights are normalized to 1 if density is True. If density is False, the values of the returned histogram are equal to the sum of the weights belonging to the samples falling into each bin.
H, edges = np.histogramdd((X, Y, Z), bins = (20, 12, 7), range=(r_x, r_y, r_z), weights = h_stress, density = False)
# histogram returns the number of samples per bin / voxel
H_2, edges_2 = np.histogramdd((X, Y, Z), range=(r_x, r_y, r_z), bins = (20, 12, 7))

In [58]:
H_2.shape

(20, 12, 7)

In [59]:
count_v = H_2.flatten()

p_6 = pv.Plotter()
p_6.add_mesh(voxels, opacity=0.15, scalars=count_v)
p_6.show()

Widget(value='<iframe src="http://localhost:53164/index.html?ui=P_0x213372551f0_17&reconnect=auto" class="pyvi…

In [64]:
bin_count=[]
stress_average = []
for i in range(len(sim_results_list)):
    sim_output = pd.read_csv(sim_results_list[i])
    X = sim_output['x']
    Y = sim_output['y']
    Z = sim_output['z']
    h_stress = sim_output['hor_stress']
    #  Weights are normalized to 1 if density is True. If density is False, the values of the returned histogram are equal to the sum of the weights belonging to the samples falling into each bin.
    H, edges = np.histogramdd((X, Y, Z), bins = (20, 12, 7), range=(r_x, r_y, r_z), weights = h_stress, density = False)
    # histogram returns the number of samples per bin / voxel
    H_2, edges_2 = np.histogramdd((X, Y, Z), range=(r_x, r_y, r_z), bins = (20, 12, 7))
    stress_sum = H.flatten()
    node_count = H_2.flatten()
    average_values = np.zeros(len(stress_sum))
    for j in range(len(average_values)):
        if node_count[j] >0:
            average_values[j]=stress_sum[j]/node_count[j]
    bin_count.append(node_count)
    stress_average.append(np.max(average_values))



In [66]:
np.shape(bin_count)

(329, 1680)

In [68]:
b_c = np.array(bin_count)

In [69]:
b_c.reshape(329, 1680)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [73]:
data = [b_c.T, stress_average]
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['NodeCount', 'Stress'])

df.to_csv('TrainingsSet1.csv', index=False)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [75]:
data = b_c
# Create the pandas DataFrame
df = pd.DataFrame(data)

df.to_csv('Dataset1.csv', index=False)

labels = stress_average
# Create the pandas DataFrame
df_l = pd.DataFrame(labels)

df_l.to_csv('Dataset1_labels.csv', index=False)